[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1hq8C0Ni0QdGMdLl0PHvdNlf3TJYyXpfV?usp=sharing)

# CRISP-DM [Amazon Product Reviews](https://www.kaggle.com/datasets/arhamrumi/amazon-product-reviews)

## 1. Bussiness Understanding

* Find positive/negative sentiment on Amazon Product Reviews 
* Build model to predict sentiment

## 2. Data Understanding

In [ ]:
from pyspark.sql.functions import *
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.tuning import *
from pyspark.ml.evaluation import *
import re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1654795038779_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
DATASET_PATH = "s3://aulia-eas/data-source/Reviews.csv"
MODEL_PATH = "s3://aulia-eas/data-output/model"
RESULT_PATH = "s3://aulia-eas/data-output/result"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df = spark.read.csv(DATASET_PATH, header=True, escape="\"")
df.show(20, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----------+--------------+-------------------------------+--------------------+----------------------+-----+----------+------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Id |ProductId |UserId        |Pro

In [ ]:
print("Dataframe consist of", df.count(), "data")
print("Dataframe consist of", len(df.columns), "features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dataframe consist of 568454 data
Dataframe consist of 10 features

In [ ]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Id: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)

All features have string data type

In [ ]:
df.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+--------------------+------------------+-----------+--------------------+----------------------+------------------+--------------------+-----------------+--------------------+
|summary|                Id|           ProductId|            UserId|ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|             Score|                Time|          Summary|                Text|
+-------+------------------+--------------------+------------------+-----------+--------------------+----------------------+------------------+--------------------+-----------------+--------------------+
|  count|            568454|              568454|            568454|     568454|              568454|                568454|            568454|              568454|           568454|              568454|
|   mean|          284227.5| 6.588314217660668E9|              null|        NaN|  1.7438174416927315|    2.2288100708236724| 4.183198640523243|1.2962566049024196E9|              NaN|  

Categorical:
* Id
* ProductId
* UserId
* ProfileName
* Score

Continuous:
* HelpfulnessNumerator
* HelpfulnessDenominator
* Time

Text:
* Summary
* Text


In [ ]:
df.groupby(['ProductId', 'UserId', 'ProfileName', 'Time', 'Text']).count().where('count > 1').sort('count', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------+-----------------+----------+--------------------+-----+
| ProductId|        UserId|      ProfileName|      Time|                Text|count|
+----------+--------------+-----------------+----------+--------------------+-----+
|B000084DWM|A3TVZM3ZIXG8YW|christopher hayes|1291420800|This review will ...|   10|
|B003WK0D8O|A3TVZM3ZIXG8YW|christopher hayes|1291420800|This review will ...|   10|
|B003M60K54|A3TVZM3ZIXG8YW|christopher hayes|1291420800|This review will ...|   10|
|B0002MLA5K|A3TVZM3ZIXG8YW|christopher hayes|1291420800|This review will ...|   10|
|B0006345PW|A3TVZM3ZIXG8YW|christopher hayes|1291420800|This review will ...|   10|
|B009GHI5Q4|A3TVZM3ZIXG8YW|christopher hayes|1291420800|This review will ...|   10|
|B003MWBFMK|A3TVZM3ZIXG8YW|christopher hayes|1291420800|This review will ...|   10|
|B001VIYCK4|A3TVZM3ZIXG8YW|christopher hayes|1291420800|This review will ...|   10|
|B001VIY8BW|A3TVZM3ZIXG8YW|christopher hayes|1291420800|This review will ...

Dataframe has duplicate data, even for different products

In [ ]:
df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---------+------+-----------+--------------------+----------------------+-----+----+-------+----+
| Id|ProductId|UserId|ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|Time|Summary|Text|
+---+---------+------+-----------+--------------------+----------------------+-----+----+-------+----+
|  0|        0|     0|         34|                   0|                     0|    0|   0|      2|   0|
+---+---------+------+-----------+--------------------+----------------------+-----+----+-------+----+

`ProfileName` and `Summary` features have null/nan values

## 3. Data Preparation

In [ ]:
# select text and score features
raw_df = df.select("Text", "Score")
raw_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|                Text|Score|
+--------------------+-----+
|I have bought sev...|    5|
|Product arrived l...|    1|
|This is a confect...|    4|
|If you are lookin...|    2|
|Great taffy at a ...|    5|
|I got a wild hair...|    4|
|This saltwater ta...|    5|
|This taffy is so ...|    5|
|Right now I'm mos...|    5|
|This is a very he...|    5|
|I don't know if i...|    5|
|One of my boys ne...|    5|
|My cats have been...|    1|
|good flavor! thes...|    4|
|The Strawberry Tw...|    5|
|My daughter loves...|    5|
|I love eating the...|    2|
|I am very satisfi...|    5|
|Twizzlers, Strawb...|    5|
|Candy was deliver...|    5|
+--------------------+-----+
only showing top 20 rows

In [ ]:
# drop duplicates data
raw_df = raw_df.dropDuplicates()
print("Number of data after removing the duplicates:", raw_df.count())
raw_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of data after removing the duplicates: 393675
+--------------------+-----+
|                Text|Score|
+--------------------+-----+
|I went to 4 groce...|    5|
|Plocky's chips ar...|    5|
|Soft smooth textu...|    5|
|I think pack of 4...|    3|
|I have fallen in ...|    5|
|This stuff makes ...|    5|
|What a great gift...|    5|
|Not much of a dec...|    3|
|We have been buyi...|    4|
|This is one of th...|    5|
|I tasted a few of...|    5|
|My favorite of th...|    5|
|The pepperoni is ...|    5|
|These are a good ...|    3|
|I love pistachio ...|    4|
|This tea is wonde...|    5|
|Crunchy.  Came pa...|    3|
|I really enjoyed ...|    2|
|I buy most everyt...|    1|
|As soon as I pick...|    5|
+--------------------+-----+
only showing top 20 rows

In [ ]:
# cast score to double
raw_df = raw_df.withColumn("Score", raw_df.Score.cast('double'))
raw_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|                Text|Score|
+--------------------+-----+
|I went to 4 groce...|  5.0|
|Plocky's chips ar...|  5.0|
|Soft smooth textu...|  5.0|
|I think pack of 4...|  3.0|
|I have fallen in ...|  5.0|
|This stuff makes ...|  5.0|
|What a great gift...|  5.0|
|Not much of a dec...|  3.0|
|We have been buyi...|  4.0|
|This is one of th...|  5.0|
|I tasted a few of...|  5.0|
|My favorite of th...|  5.0|
|The pepperoni is ...|  5.0|
|These are a good ...|  3.0|
|I love pistachio ...|  4.0|
|This tea is wonde...|  5.0|
|Crunchy.  Came pa...|  3.0|
|I really enjoyed ...|  2.0|
|I buy most everyt...|  1.0|
|As soon as I pick...|  5.0|
+--------------------+-----+
only showing top 20 rows

In [ ]:
# define bucket borders (splits array), 0.0: -1.0 < x < 3.0 and 1.0: 3.0 <= x < 6.0
bucketBorders = [-1.0, 3.0, 6.0] 
# initialize Bucketizer with defined bucket borders as splits, Score as input column, and Sentiment as output column
bucketer = Bucketizer().setSplits(bucketBorders).setInputCol("Score").setOutputCol("Sentiment")
# apply it to raw_df
raw_df = bucketer.transform(raw_df)
raw_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+---------+
|                Text|Score|Sentiment|
+--------------------+-----+---------+
|I went to 4 groce...|  5.0|      1.0|
|Plocky's chips ar...|  5.0|      1.0|
|Soft smooth textu...|  5.0|      1.0|
|I think pack of 4...|  3.0|      1.0|
|I have fallen in ...|  5.0|      1.0|
|This stuff makes ...|  5.0|      1.0|
|What a great gift...|  5.0|      1.0|
|Not much of a dec...|  3.0|      1.0|
|We have been buyi...|  4.0|      1.0|
|This is one of th...|  5.0|      1.0|
|I tasted a few of...|  5.0|      1.0|
|My favorite of th...|  5.0|      1.0|
|The pepperoni is ...|  5.0|      1.0|
|These are a good ...|  3.0|      1.0|
|I love pistachio ...|  4.0|      1.0|
|This tea is wonde...|  5.0|      1.0|
|Crunchy.  Came pa...|  3.0|      1.0|
|I really enjoyed ...|  2.0|      0.0|
|I buy most everyt...|  1.0|      0.0|
|As soon as I pick...|  5.0|      1.0|
+--------------------+-----+---------+
only showing top 20 rows

In [ ]:
def text_cleaning(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"http\S+|www\.\S+|<.*?>|[^a-zA-Z\s]", " ", text) # remove url, html tag, and non words
    text = re.sub(r"\s\s+", " ", text) # strip whitespaces
    return text.strip()
clean_text = udf(lambda text: text_cleaning(text))

# clean text from url, html tag, and non words
raw_df = raw_df.withColumn("Text", clean_text(col("Text")))
raw_df.show(20, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+---------+
|Text                                                                                                                                                                                                                                                                                                                              

In [ ]:
# initialize tokenizer (split sentences into words)
tokenizer = Tokenizer().setInputCol("Text").setOutputCol("Words")

# initialize stopWordsRemover (remove englist stopwords)
englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
stopWordsRemover = StopWordsRemover().setStopWords(englishStopWords).setInputCol(tokenizer.getOutputCol()).setOutputCol("FilteredWords")

# initialize TF-IDF (hash each word, convert it to numerical representation, and weigh each word in the voculary according to the inverse document frequency)
tf = HashingTF().setInputCol(stopWordsRemover.getOutputCol()).setOutputCol("TFOut").setNumFeatures(1000)
idf = IDF().setInputCol(tf.getOutputCol()).setOutputCol("IDFOut").setMinDocFreq(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# split data into train_df and test_df
train_df, test_df = raw_df.select(["Sentiment", "Text"]).randomSplit([0.8, 0.2])
train_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+
|Sentiment|                Text|
+---------+--------------------+
|      0.0|A couple years ag...|
|      0.0|A friend gave me ...|
|      0.0|A little too eart...|
|      0.0|A majority of the...|
|      0.0|A number of us ea...|
|      0.0|A shop in my neig...|
|      0.0|ASIN I purchase t...|
|      0.0|According to the ...|
|      0.0|After about a yea...|
|      0.0|After finding out...|
|      0.0|After finding out...|
|      0.0|After having REAL...|
|      0.0|After having my v...|
|      0.0|After reading all...|
|      0.0|After reading rev...|
|      0.0|After reading rev...|
|      0.0|After reading the...|
|      0.0|After reading the...|
|      0.0|After spending on...|
|      0.0|After the disappo...|
+---------+--------------------+
only showing top 20 rows

In [ ]:
# initialize pipeline and fit it to train_df
pipeline = Pipeline(stages=[tokenizer, stopWordsRemover, tf, idf])
fittedPipeline = pipeline.fit(train_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# apply fittedPipeline to train_df
train_df = fittedPipeline.transform(train_df)
train_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Sentiment|                Text|               Words|       FilteredWords|               TFOut|              IDFOut|
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      0.0|A couple years ag...|[a, couple, years...|[couple, years, a...|(1000,[80,133,157...|(1000,[80,133,157...|
|      0.0|A friend gave me ...|[a, friend, gave,...|[friend, gave, ba...|(1000,[3,12,43,55...|(1000,[3,12,43,55...|
|      0.0|A little too eart...|[a, little, too, ...|[little, earthy, ...|(1000,[34,50,144,...|(1000,[34,50,144,...|
|      0.0|A majority of the...|[a, majority, of,...|[majority, pods, ...|(1000,[60,88,89,1...|(1000,[60,88,89,1...|
|      0.0|A number of us ea...|[a, number, of, u...|[number, us, earn...|(1000,[29,61,111,...|(1000,[29,61,111,...|
|      0.0|A shop in my neig...|[a, shop, in, my,...|[shop, neig

In [ ]:
# apply fittedPipeline to test_df
test_df = fittedPipeline.transform(test_df)
test_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Sentiment|                Text|               Words|       FilteredWords|               TFOut|              IDFOut|
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      0.0|A big lover of il...|[a, big, lover, o...|[big, lover, illy...|(1000,[23,38,44,4...|(1000,[23,38,44,4...|
|      0.0|A couple of weeks...|[a, couple, of, w...|[couple, weeks, t...|(1000,[10,26,48,1...|(1000,[10,26,48,1...|
|      0.0|A short backgroun...|[a, short, backgr...|[short, backgroun...|(1000,[38,73,89,9...|(1000,[38,73,89,9...|
|      0.0|A warning should ...|[a, warning, shou...|[warning, placed,...|(1000,[9,14,30,48...|(1000,[9,14,30,48...|
|      0.0|AM I CRAZY OR ARE...|[am, i, crazy, or...|[crazy, really, c...|(1000,[51,112,139...|(1000,[51,112,139...|
|      0.0|AVOID AT ALL COST...|[avoid, at, all, ...|[avoid, cos

## 4. Modeling

LogisticRegression:
* Fast
* Higher accuracy for high dimensional data

In [ ]:
lr = LogisticRegression(labelCol="Sentiment", featuresCol="IDFOut")
grid = ParamGridBuilder() \
  .addGrid(lr.elasticNetParam , [0.0, 0.1, 0.2]) \
  .addGrid(lr.regParam, [0.1, 0.2, 0.3]) \
  .addGrid(lr.maxIter, [5, 10, 20]) \
  .build()
evaluator = BinaryClassificationEvaluator(labelCol="Sentiment", metricName="areaUnderROC", numBins=0)
crossValidator = CrossValidator(estimator=lr, 
                                evaluator=evaluator, 
                                estimatorParamMaps=grid, 
                                numFolds=5)
cv = crossValidator.fit(train_df)

In [ ]:
best_model = cv.bestModel

In [ ]:
print(best_model._java_obj.getElasticNetParam())
print(best_model._java_obj.getRegParam())
print(best_model._java_obj.getMaxIter())

0.0
0.1
10


In [ ]:
lr = LogisticRegression(labelCol="Sentiment", featuresCol="IDFOut", elasticNetParam=0.0, regParam=0.1, maxIter=10)
best_model = lr.fit(train_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 5. Evaluation

In [ ]:
prediction = best_model.transform(test_df)
prediction.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Sentiment|                Text|               Words|       FilteredWords|               TFOut|              IDFOut|       rawPrediction|         probability|prediction|
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|      0.0|A big lover of il...|[a, big, lover, o...|[big, lover, illy...|(1000,[23,38,44,4...|(1000,[23,38,44,4...|[-2.6005511799337...|[0.06910295576337...|       1.0|
|      0.0|A couple of weeks...|[a, couple, of, w...|[couple, weeks, t...|(1000,[10,26,48,1...|(1000,[10,26,48,1...|[-0.4964367489271...|[0.37837841069286...|       1.0|
|      0.0|A short backgroun...|[a, short, backgr...|[short, backgroun...|(1000,[38,73,89,9...|(1000,[38,73,89,9...|[-1.2063476602477...|[0.2303479282

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="Sentiment", metricName="areaUnderROC", numBins=0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
print(evaluator.evaluate(prediction))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8634534587963756

## 6. Deployment


In [ ]:
best_model.save(MODEL_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
prediction.select(["Text", "Sentiment", "prediction"]).write.mode("overwrite").csv(RESULT_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…